In [ ]:
import requests # requests: HTTP 요청을 보내기 위해 사용하는 라이브러리이고 API와의 통신에 필요함.
from langchain import LLMChain # langchain: 자연어 처리와 관련된 다양한 기능을 제공하는 라이브러리. 여기서는 LLM(대형 언어 모델)과 프롬프트 템플릿을 사용합니다.
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate #LLMChain, OpenAI, PromptTemplate: LangChain에서 제공하는 클래스들로 OpenAI 모델을 사용하여 텍스트 생성 및 응답을 구성하는 데 사용됩니다.

# 지하철 혼잡도 데이터 API (가상의 URL)
#API_URL = "https://api.subway.com/congestion"
API_URL = "http://data.seoul.go.kr/dataList/262/S/2/datasetView.do" #API_URL: API에 요청을 보낼 URL을 설정합니다. 실제로는 서울시 데이터 포털의 URL이 설정되어 있습니다. 

# 혼잡도 데이터를 가져오는 함수
def get_congestion_data(station): #get_congestion_data(station): 주어진 역 이름(station)을 사용하여 API에 요청을 보내고 응답을 반환하는 함수임.
    response = requests.get(f"{API_URL}?station={station}") #requests.get(...): 지정된 URL로 GET 요청을 보내고 station 변수가 쿼리 파라미터로 추가된다. 
    if response.status_code == 200: #response.status_code: API의 응답 상태 코드를 확인하고 200이면 성공적으로 데이터를 가져온 것이고 그렇지 않으면 오류 메시지를 출력한다.
        return response.json() #response.json(): 응답을 JSON 형식으로 변환하여 반환하고 오류가 발생하면 상태 코드를 출력하고 None을 반환한다.
    else:
        print(f"응답 상태 코드: {response.status_code}")  # 상태 코드 출력
        return None

# 챗봇을 위한 프롬프트 템플릿 설정
prompt_template = PromptTemplate( #PromptTemplate: OpenAI 모델에 전달할 프롬프트를 정의하는 클래스.
    input_variables=["congestion_info"], #input_variables: 템플릿에서 사용할 변수를 지정함. 여기서는 congestion_info 변수를 사용.
    template="현재 {congestion_info['station']}역의 혼잡도는 {congestion_info['congestion_level']}입니다." #template: 실제로 모델에 전달할 문자열 템플릿을 정의한다. {congestion_info['station']} 및 {congestion_info['congestion_level']} 부분은 실제 데이터로 대체된다.
)

# OpenAI LLM 설정
llm = OpenAI(model="gpt-3.5-turbo") #OpenAI: OpenAI의 모델을 사용할 객체를 생성한다. 여기서는 gpt-3.5-turbo 모델을 사용하고 있고 이 모델은 자연어 응답을 생성하는 데 사용된다.



# 챗봇 응답 생성 함수
def generate_response(station): #generate_response(station): 사용자가 입력한 역 이름을 받아 혼잡도 데이터를 가져오고 그에 대한 응답을 생성하는 함수임.
    congestion_data = get_congestion_data(station) #congestion_data = get_congestion_data(station): 입력된 역 이름을 사용하여 혼잡도 데이터를 요청함.
    if congestion_data:
        response = LLMChain(llm=llm, prompt=prompt_template).run(congestion_info=congestion_data) #LLMChain(...).run(...): 가져온 데이터를 사용하여 OpenAI 모델을 통해 최종 응답을 생성함.
        return response
    else:
        return "데이터를 가져오는 데 실패했습니다." #데이터가 성공적으로 가져와지지 않으면 오류 메시지를 반환함.

# 사용자 입력 받기 (예시)
station_name = input("어느 역의 혼잡도를 알고 싶으신가요? ") #input(...): 사용자에게 지하철 역 이름을 입력하라는 메시지를 출력하고 입력된 값을 station_name 변수에 저장함.
response = generate_response(station_name) #generate_response(station_name): 사용자가 입력한 역 이름을 사용하여 혼잡도 데이터를 요청하고 응답을 생성함.
print(response) #print(response): 최종 응답을 출력합니다.


응답 상태 코드: 400
데이터를 가져오는 데 실패했습니다.


이 코드는 사용자가 지하철 역 이름을 입력하면, 해당 역의 혼잡도를 API를 통해 조회하고, 그 결과를 자연어 형식으로 출력하는 챗봇의 기능을 수행합니다. 다만, 실제 API URL과 데이터 구조에 맞게 코드를 조정해야 정상적으로 작동할 수 있습니다. 